In [1]:
import numpy as np
from tqdm.notebook import tqdm

# The below approach is extremely slow, so ignore it.
def _matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [2]:
import cupy as cp
from tqdm import tqdm

def get_batches(R, batch_size):
    """Generate mini-batches for the non-zero entries in R."""
    #Old approach that replaces nan with 0 and then filters out the 0 entries
    #R_without_nan = np.nan_to_num(R, nan=0)
    #non_zero_indices = cp.asarray(R_without_nan).nonzero()
    
    #Get the non-nan indices
    nan_mask = np.isnan(R)
    non_nan_indices = np.where(~nan_mask)
    
    num_samples = len(non_nan_indices[0])
    indices = cp.arange(num_samples)
    cp.random.shuffle(indices)
    
    
    for idx in range(0, num_samples, batch_size):
        batch_indices = indices[idx:idx+batch_size]
        i_indices = non_nan_indices[0][batch_indices]
        j_indices = non_nan_indices[1][batch_indices]
        values = R[i_indices, j_indices]
        yield i_indices, j_indices, values

def matrix_factorization_gpu_batched(R, P, Q, K, batch_size=1000, steps=5000, alpha=0.0002, beta=0.02):
    R = cp.array(R)
    P = cp.array(P)
    Q = cp.array(Q)
    Q = Q.T
    
    for step in tqdm(range(steps), desc="Epochs"):
        for i_indices, j_indices, values in tqdm(get_batches(R, batch_size), desc="Rows", leave=False):
            errors = values - cp.sum(P[i_indices] * Q[:, j_indices].T, axis=1)
            
            for k in range(K):
                P[i_indices, k] += alpha * (2 * errors * Q[k, j_indices] - beta * P[i_indices, k])
                Q[k, j_indices] += alpha * (2 * errors * P[i_indices, k] - beta * Q[k, j_indices])

        eR = cp.dot(P, Q)
        e = cp.sum((R[R > 0] - eR[R > 0]) ** 2) + beta/2.0 * (cp.sum(P**2) + cp.sum(Q**2))
        print(e)
        #l += 1
        
        if e < 0.001:
            break

    return cp.asnumpy(P), cp.asnumpy(Q.T)

In [3]:
import pandas as pd
DATA_DIR = 'data'
train_ratings = pd.read_csv(f'{DATA_DIR}/train_ratings.csv')

In [4]:
#how many unique movies and users do we have?
n_movies = train_ratings.movieId.unique().shape[0]
n_users = train_ratings.userId.unique().shape[0]
print(f'Number of unique movies: {n_movies}')
print(f'Number of unique users: {n_users}')

Number of unique movies: 8983
Number of unique users: 610


In [5]:
train_matrix = train_ratings.pivot_table(index='userId', columns='movieId', values='rating')

In [6]:
N = len(train_matrix)
M = len(train_matrix.columns)
N, M

(610, 8983)

In [7]:
K = 10

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization_gpu_batched(train_matrix.values, P, Q, K, steps=2000)

Epochs:   0%|                                                                                 | 0/2000 [00:00<?, ?it/s]
Rows: 0it [00:00, ?it/s]
Rows: 1it [00:00,  4.34it/s]
Rows: 14it [00:00, 52.07it/s]
Rows: 28it [00:00, 82.79it/s]
Rows: 40it [00:00, 95.00it/s]
Rows: 52it [00:00, 98.70it/s]
Rows: 63it [00:00, 101.45it/s]
Rows: 76it [00:00, 108.27it/s]
Epochs:   0%|                                                                         | 1/2000 [00:01<39:45,  1.19s/it]

187227.032504817



Rows: 0it [00:00, ?it/s]
Rows: 11it [00:00, 103.06it/s]
Rows: 25it [00:00, 122.98it/s]
Rows: 38it [00:00, 112.25it/s]
Rows: 51it [00:00, 116.26it/s]
Rows: 65it [00:00, 121.91it/s]
Rows: 79it [00:00, 125.41it/s]
Epochs:   0%|                                                                         | 2/2000 [00:01<30:16,  1.10it/s]

170208.14672279646



Rows: 0it [00:00, ?it/s]
Rows: 10it [00:00, 91.16it/s]
Rows: 20it [00:00, 73.46it/s]
Rows: 28it [00:00, 73.50it/s]
Rows: 38it [00:00, 80.78it/s]
Rows: 47it [00:00, 83.31it/s]
Rows: 56it [00:00, 83.39it/s]
Rows: 65it [00:00, 85.24it/s]
Rows: 75it [00:00, 89.51it/s]
Epochs:   0%|                                                                         | 3/2000 [00:02<31:27,  1.06it/s]

...

24754.08428646524



Rows: 0it [00:00, ?it/s]
Rows: 10it [00:00, 91.12it/s]
Rows: 20it [00:00, 90.19it/s]
Rows: 30it [00:00, 90.57it/s]
Rows: 40it [00:00, 93.82it/s]
Rows: 52it [00:00, 102.28it/s]
Rows: 63it [00:00, 103.65it/s]
Rows: 74it [00:00, 105.41it/s]
Epochs: 100%|█████████████████████████████████████████████████████████████████████▉| 1999/2000 [23:40<00:00,  1.41it/s]

24750.678915333672



Rows: 0it [00:00, ?it/s]
Rows: 12it [00:00, 115.13it/s]
Rows: 26it [00:00, 126.05it/s]
Rows: 39it [00:00, 126.75it/s]
Rows: 53it [00:00, 128.56it/s]
Rows: 67it [00:00, 130.84it/s]
Rows: 81it [00:00, 131.38it/s]
Epochs: 100%|██████████████████████████████████████████████████████████████████████| 2000/2000 [23:41<00:00,  1.41it/s]

24746.16532100533


In [8]:
nP.shape, nQ.shape

((610, 10), (8983, 10))

In [9]:
nQ

array([[ 0.63450927,  0.85654306,  0.65818797, ...,  0.19008813,
         0.5134116 ,  0.69879064],
       [ 0.34768656,  0.40598347,  0.49836665, ...,  0.46967023,
         0.71642437,  0.77808946],
       [-0.22353797,  0.57741599,  0.39393685, ...,  0.04164827,
         0.76563461,  0.97677542],
       ...,
       [ 0.85345814,  0.94871537,  0.67282657, ...,  0.72305286,
         0.7026309 ,  0.41500624],
       [ 0.19994594,  0.01864869,  0.75803445, ...,  0.41050628,
         0.20083973,  0.78756768],
       [ 1.0068277 ,  0.75237358,  0.30636589, ...,  0.72870012,
         0.77000691,  0.39064741]])

In [13]:
import pandas as pd

# Load the test set
test_set = pd.read_csv(f'{DATA_DIR}/test_set_no_ratings.csv')

# Placeholder list to store the predicted ratings
predicted_ratings = []

# Iterate over each row in the test set
for _, row in test_set.iterrows():
    userId = row['userId']
    movieId = row['movieId']
    
    
    # Check if userId and movieId exist in ratings_matrix
    if userId in train_matrix.index and movieId in train_matrix.columns:
        user_idx = list(train_matrix.index).index(userId)
        movie_idx = list(train_matrix.columns).index(movieId)
        
        predicted_rating = np.dot(nP[user_idx, :], nQ[movie_idx, :].T)
    else:
        # Use a default value for unknown userIds or movieIds
        predicted_rating = np.mean(train_matrix.values[train_matrix.values > 0])
    
    predicted_ratings.append(predicted_rating)

In [14]:
train_matrix.shape

(610, 8983)

In [15]:
# Add the predicted ratings to the test set dataframe and save to CSV
test_set['rating'] = predicted_ratings
predictions_df = test_set[['Id', 'rating']]
predictions_df.to_csv('predictions.csv', index=False)